In [5]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import math

In [6]:
# load attributes 
data_dir = os.getcwd() + '\\Data\\'
train_data = pd.read_csv(data_dir + 'train.csv')
train_labels = pd.read_csv(data_dir + 'train_labels.csv')
train_label_bits = np.genfromtxt(data_dir + 'train_label_bits.csv', delimiter=',', dtype=np.float32) 

# loading revised images
image_dir = data_dir + 'images\\res_rev_05\\' 

# total count of samples
sample_count = 1584

# load images
images = []
for image_label in range(sample_count):
    image = mpimg.imread(image_dir + str(image_label + 1) + '.jpg')
    images.append(image)
    
images = np.array(images)

train_images = images[train_data['id'] - 1]
train_images = np.reshape(train_images, (len(train_data), images[0].shape[0], images[0].shape[1], -1))
train_images.shape

(990, 85, 54, 1)

In [7]:
TOTAL_COUNT = len(images)
TRAIN_COUNT = len(train_data) # training image count in input csv file
IMAGE_WIDTH  = images[0].shape[0] # input image width
IMAGE_HEIGHT = images[0].shape[1] # input image height
OUTPUT_TYPE_COUNT = len(list(set(train_data['species'])))
print('Image Count: ', TOTAL_COUNT)
print('Image Size: ', IMAGE_WIDTH , 'X' , IMAGE_HEIGHT)
print('Train Count: ', TRAIN_COUNT)
print('Specy Count: ', OUTPUT_TYPE_COUNT)

Image Count:  1584
Image Size:  85 X 54
Train Count:  990
Specy Count:  99


In [8]:
ITERATION_COUNT = 200
ACCURACY_CHECK = 20
BATCH_SIZE = 30

In [9]:
# the batch size equals to BATCH_SIZE it will
# starts from 0 to until the last index of
# batch equals to TOTAL_COUNT
BATCH_COUNTER = 0

def next_batch(all_images, all_labels):
    
    # accessing BATCH_COUNTER inside function
    global BATCH_COUNTER
    
    # check if exceeds to the last index then reset batch_counter
    if BATCH_COUNTER == len(all_labels) / BATCH_SIZE:
        BATCH_COUNTER = 0

    # setting first and last index of data
    index_from = BATCH_COUNTER * BATCH_SIZE
    index_to   = (BATCH_COUNTER + 1) * BATCH_SIZE
    
    # loading a batch of training images and labels
    image_batch = all_images[index_from:index_to]
    label_batch = all_labels[index_from:index_to]
    
    # incrementing batch_counter for the next iteration
    BATCH_COUNTER += 1 
    
    return image_batch, label_batch;

In [21]:
C0 = 1   # input channel count
C1 = 4   # convolutional network channel 1 count 
C2 = 8   # convolutional network channel 2 count
C3 = 12  # convolutional network channel 3 count
C4 = 200 # fulley connected layer size
C5 = OUTPUT_TYPE_COUNT  # output count 

# weights
W1 = tf.Variable(tf.truncated_normal([5, 5, C0, C1], stddev = 0.1)) 
W2 = tf.Variable(tf.truncated_normal([5, 5, C1, C2], stddev = 0.1))
W3 = tf.Variable(tf.truncated_normal([4, 4, C2, C3], stddev = 0.1))
W4 = tf.Variable(tf.truncated_normal([22 * 14 * C3, C4], stddev = 0.1))
W5 = tf.Variable(tf.truncated_normal([C4, C5], stddev = 0.1))
    
# biases
B1 = tf.Variable(tf.ones([C1]) / 10)
B2 = tf.Variable(tf.ones([C2]) / 10)
B3 = tf.Variable(tf.ones([C3]) / 10)
B4 = tf.Variable(tf.ones([C4]) / 10)
B5 = tf.Variable(tf.ones([C5]) / 10)

# model
stride = 1  # output is 85x54
X   = tf.placeholder(tf.float32, shape=[None, IMAGE_WIDTH, IMAGE_HEIGHT, C0])
Y1  = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
stride = 2  # output is 43x27
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
stride = 2  # output is 22x14
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)
    
# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y3, shape=[-1, 22 * 14 * C3])
Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)
                      
# paceholder for learing_rate variable
lr = tf.placeholder(tf.float32)
    
# placeholder for correct answers
Y_ = tf.placeholder(tf.float32, [None, C5])
    
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * 100
    
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# training step, the learning rate is a placeholder
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

# TensorBoard initialization
tf.summary.scalar('cost', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('/log/tb')

# initialisation
tf.set_random_seed(0)
init = tf.global_variables_initializer()

# start computation
sess = tf.Session()
sess.run(init)
    
for i in range(ITERATION_COUNT):
    
    # learning rate decay
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
        
    # load batch of images and correct answers
    batch_X , batch_Y = next_batch(train_images, train_label_bits)
        
    # train
    sess.run(train_step, feed_dict = {X: batch_X, Y_: batch_Y, lr: learning_rate})
    
    if (i + 1) % ACCURACY_CHECK == 0:
        batch_X , batch_Y = next_batch(train_images, train_label_bits)
        a , c = sess.run([accuracy, cross_entropy], feed_dict = {X: batch_X, Y_: batch_Y})
        writer.add_summary(summary, i)
        print(i + 1, ":" , a, c, learning_rate)
    
    
sess.close()

20 : 0.0 472.541 0.0029725804490844114
40 : 0.2 424.526 0.002943997796047132
60 : 0.266667 318.866 0.0029156995451594635
80 : 0.566667 184.846 0.002887682866572735
100 : 0.933333 43.9806 0.0028599449585957402
120 : 0.7 83.8761 0.0028324830474145673
140 : 0.966667 24.8447 0.0028052943868152125
160 : 1.0 6.7887 0.002778376257908959
180 : 1.0 6.25102 0.0027517259688604836
200 : 1.0 1.24231 0.0027253408546186733
